In [5]:
import torch
import numpy as np
device = "cuda"
voxel_size = 0.03

In [2]:
points = np.load("ray_points_camera_1111.npy") #(7500,50,3)
# 改
occupancy_mask = np.load("pred_outcome_for_npy_and_ply/predicted_npy_pw_0.64_origin_lr_0.0008_bs_16_epoch_28/a5af701c-1825-40ce-a9d7-0b7ed08fb43b_52_predicted.npy")
print("占用掩码的形状:", occupancy_mask.shape) #(100,75,50)
print("占用掩码的和:", occupancy_mask.sum())
# query_points = np.load("knn_test/regular_cube_from_ray_to_npy.npy")
# print("查询点云数据的形状:", query_points.shape)

占用掩码的形状: (100, 75, 50)
占用掩码的和: 20855


In [ ]:
points.dtype

In [3]:
point_cloud = torch.tensor(points.reshape(-1, 3), dtype=torch.float32, device=device)  # (N, 3)

In [ ]:
# occupancy_mask = torch.tensor(occupancy_mask, dtype=torch.float32)
# occupancy_mask_flat = occupancy_mask.flatten()
# occupancy_mask_flat.sum()

In [4]:
# 将数据转为 tensor
mask = torch.tensor(occupancy_mask, dtype=torch.float32, device=device)

In [6]:
# 计算点云范围
min_coords = torch.min(point_cloud, dim=0)[0]
max_coords = torch.max(point_cloud, dim=0)[0]
# 生成规则 voxel 的形状
grid_size = ((max_coords - min_coords) / voxel_size).ceil().int()  # 每个维度的体素数
D, H, W = grid_size

In [7]:
print(D, H, W)

tensor(84, device='cuda:0', dtype=torch.int32) tensor(69, device='cuda:0', dtype=torch.int32) tensor(66, device='cuda:0', dtype=torch.int32)


In [9]:
# 生成规则网格坐标
grid_x = torch.linspace(min_coords[0], max_coords[0], W, device=device)
grid_y = torch.linspace(min_coords[1], max_coords[1], H, device=device)
grid_z = torch.linspace(min_coords[2], max_coords[2], D, device=device)
grid_z, grid_y, grid_x = torch.meshgrid(grid_z, grid_y, grid_x, indexing="ij")
grid_coords = torch.stack([grid_x, grid_y, grid_z], dim=-1)  # (D, H, W, 3)
grid_coords.shape

torch.Size([84, 69, 66, 3])

In [12]:
# 将点云映射到规则网格的范围
normalized_coords = (grid_coords - min_coords) / (max_coords - min_coords) * 2 - 1  # [-1, 1] 归一化
normalized_coords = normalized_coords.unsqueeze(0)  # 添加批次维度以供 affine_grid 使用
normalized_coords.shape

torch.Size([1, 84, 69, 66, 3])

In [13]:
# 使用 affine_grid 进行插值
mask = mask.unsqueeze(0).unsqueeze(0)  # 添加批次和通道维度
mask.shape

torch.Size([1, 1, 100, 75, 50])

In [ ]:

voxel_grid = torch.nn.functional.grid_sample(mask, normalized_coords, mode="bilinear", padding_mode="zeros", align_corners=True)

In [ ]:
import torch
import numpy as np

def point_cloud_to_voxel(point_cloud, mask, voxel_size=0.04, device="cuda"):
    """
    将不规则的点云转换为规则 voxel 网格。
    
    参数:
        point_cloud: (N, 3) 的点云数据 (x, y, z)。
        mask: (width, height, depth) 的掩码，表示点云中点的占据状态。
        voxel_size: float，目标 voxel 的间距大小。
        device: 使用的设备，默认 'cuda'。
    
    返回:
        voxel_grid: (D, H, W) 的规则 voxel 数据。
    """
    # 将数据转为 tensor
    point_cloud = torch.tensor(point_cloud, dtype=torch.float32, device=device)
    mask = torch.tensor(mask, dtype=torch.float32, device=device)

    # 计算点云范围
    min_coords = torch.min(point_cloud, dim=0)[0]
    max_coords = torch.max(point_cloud, dim=0)[0]

    # 生成规则 voxel 的形状
    grid_size = ((max_coords - min_coords) / voxel_size).ceil().int()  # 每个维度的体素数
    D, H, W = grid_size

    # 生成规则网格坐标
    grid_x = torch.linspace(min_coords[0], max_coords[0], W, device=device)
    grid_y = torch.linspace(min_coords[1], max_coords[1], H, device=device)
    grid_z = torch.linspace(min_coords[2], max_coords[2], D, device=device)
    grid_z, grid_y, grid_x = torch.meshgrid(grid_z, grid_y, grid_x, indexing="ij")
    grid_coords = torch.stack([grid_x, grid_y, grid_z], dim=-1)  # (D, H, W, 3)

    # 将点云映射到规则网格的范围
    normalized_coords = (grid_coords - min_coords) / (max_coords - min_coords) * 2 - 1  # [-1, 1] 归一化
    normalized_coords = normalized_coords.unsqueeze(0)  # 添加批次维度以供 affine_grid 使用

    # 使用 affine_grid 进行插值
    mask = mask.unsqueeze(0).unsqueeze(0)  # 添加批次和通道维度
    voxel_grid = torch.nn.functional.grid_sample(mask, normalized_coords, mode="bilinear", padding_mode="zeros", align_corners=True)

    return voxel_grid[0, 0]  # 返回去掉批次和通道维度的 voxel 数据


In [ ]:
# 计算点云的最小值和最大值
min_point = flattened_points.min(dim=0).values
max_point = flattened_points.max(dim=0).values
min_point

In [ ]:
# 确定体素的间距和体素网格的大小
voxel_spacing = 0.03
voxel_resolution = ((max_point - min_point) / voxel_spacing).ceil().long()
voxel_resolution

In [ ]:
# 计算点云的标准化坐标
normalized_points = (flattened_points - min_point) / (max_point - min_point)
normalized_points.shape #torch.Size([375000, 3])

In [ ]:
normalized_points * (voxel_resolution - 1)

In [ ]:
# 将标准化后的坐标映射到体素网格
voxel_indices = (normalized_points * (voxel_resolution - 1)).int()
voxel_indices.shape

In [21]:
# 确保索引在有效范围内
voxel_indices = torch.clamp(voxel_indices, min=torch.tensor(0), max=voxel_resolution - 1)

In [31]:
# 初始化一个新的规则网格
new_voxel_grid = torch.zeros(voxel_resolution[0], voxel_resolution[1], voxel_resolution[2], dtype=torch.float32)

# 计算每个点的占据信息
# 用高级索引将每个点的占据信息映射到体素网格
voxel_indices_flattened = voxel_indices[:, 0] * (voxel_resolution[1] * voxel_resolution[2]) + \
                          voxel_indices[:, 1] * voxel_resolution[2] + \
                          voxel_indices[:, 2]

In [ ]:
voxel_indices_flattened.shape

In [ ]:
# 创建规则网格
new_voxel_grid = torch.zeros(voxel_resolution.prod(), dtype=torch.float32)  # 一维规则网格

# 将占据信息映射到规则网格 (处理索引冲突)
new_voxel_grid.scatter_(0, voxel_indices_flattened, occupancy_mask_flat, reduce='add')

In [ ]:
new_voxel_grid.shape

In [ ]:
# 恢复为三维规则网格
new_voxel_grid = new_voxel_grid.view(*voxel_resolution)
new_voxel_grid.sum()

In [ ]:
new_voxel_grid.max()

In [ ]:
# 获取映射后的点云
mapped_points = torch.nonzero(new_voxel_grid) * 0.03

In [ ]:
new_voxel_grid_flat = new_voxel_grid.flatten()
new_voxel_grid_flat.shape

In [ ]:
# 将占据信息映射到规则网格
new_voxel_grid_flat = new_voxel_grid.flatten()
new_voxel_grid_flat[voxel_indices_flattened] = occupancy_mask.view(-1)

# 获取映射后的体素网格
new_voxel_grid = new_voxel_grid_flat.view(voxel_resolution)

In [ ]:
# 将点云和掩码转换为 PyTorch 张量
flattened_points = torch.tensor(points.reshape(-1, 3), dtype=torch.float32)  # (N, 3)
# query_points = torch.tensor(query_points, dtype=torch.float32, device='cuda')  # (M, 3)
flattened_mask = torch.tensor(occupancy_mask.ravel(), dtype=torch.float32, device='cuda')  # (N,)

## 1125


In [4]:
import numpy as np
import torch

def generate_regular_cube_points_from_ray(ray_points_camera, spacing=0.03):
    """
    基于 ray_points_camera 生成一个规则的三维网格，确保网格覆盖 ray_points_camera 的范围。
    
    :param ray_points_camera: (N, max_steps, 3) 形状的张量，表示光线的坐标
    :param spacing: 每个点之间的间距
    :return: 点云数据，形状为 (M, 3)，其中 M 是网格点的数量
    """
    # 获取 ray_points_camera 的最小值和最大值，确定立方体的尺寸范围
    # Find min/max along each axis (axis 0 is the rays, axis 1 is the points)
    min_coords_x = ray_points_camera[:, :, 0].min()
    min_coords_y = ray_points_camera[:, :, 1].min()
    min_coords_z = ray_points_camera[:, :, 2].min()

    max_coords_x = ray_points_camera[:, :, 0].max()
    max_coords_y = ray_points_camera[:, :, 1].max()
    max_coords_z = ray_points_camera[:, :, 2].max()

    # Create min and max coordinates
    min_coords = torch.tensor([min_coords_x, min_coords_y, min_coords_z])
    max_coords = torch.tensor([max_coords_x, max_coords_y, max_coords_z])

    # 计算立方体的尺寸（宽度、高度、深度）
    dimensions = max_coords - min_coords

    # 计算每个维度的步数
    x_steps = int(np.ceil(dimensions[0] / spacing))
    y_steps = int(np.ceil(dimensions[1] / spacing))
    z_steps = int(np.ceil(dimensions[2] / spacing))
    print(f"x_steps: {x_steps}, y_steps: {y_steps}, z_steps: {z_steps}")

    # 生成规则的三维网格
    x = np.linspace(min_coords[0], max_coords[0], x_steps)
    y = np.linspace(min_coords[1], max_coords[1], y_steps)
    z = np.linspace(min_coords[2], max_coords[2], z_steps)
    
    # 创建网格点
    x_grid, y_grid, z_grid = np.meshgrid(x, y, z)
    
    # 将网格转换为点云坐标
    points = np.vstack([x_grid.ravel(), y_grid.ravel(), z_grid.ravel()]).T
    print(f"points shape: {points.shape}")
    return points

In [8]:
ray_points_camera = np.load("ray_points_camera_1111.npy")
print(ray_points_camera.shape)
spacing = 0.03

(7500, 50, 3)


In [7]:
# 获取 ray_points_camera 的最小值和最大值，确定立方体的尺寸范围
# Find min/max along each axis (axis 0 is the rays, axis 1 is the points)
min_coords_x = ray_points_camera[:, :, 0].min()
min_coords_y = ray_points_camera[:, :, 1].min()
min_coords_z = ray_points_camera[:, :, 2].min()
max_coords_x = ray_points_camera[:, :, 0].max()
max_coords_y = ray_points_camera[:, :, 1].max()
max_coords_z = ray_points_camera[:, :, 2].max()
# Create min and max coordinates
min_coords = torch.tensor([min_coords_x, min_coords_y, min_coords_z])
max_coords = torch.tensor([max_coords_x, max_coords_y, max_coords_z])
# 计算立方体的尺寸（宽度、高度、深度）
dimensions = max_coords - min_coords
dimensions

tensor([2.4946, 2.0575, 1.9599])

In [9]:
# 计算每个维度的步数
x_steps = int(np.ceil(dimensions[0] / spacing))
y_steps = int(np.ceil(dimensions[1] / spacing))
z_steps = int(np.ceil(dimensions[2] / spacing))
print(f"x_steps: {x_steps}, y_steps: {y_steps}, z_steps: {z_steps}")
# 生成规则的三维网格
x = np.linspace(min_coords[0], max_coords[0], x_steps)
y = np.linspace(min_coords[1], max_coords[1], y_steps)
z = np.linspace(min_coords[2], max_coords[2], z_steps)
    
# 创建网格点
x_grid, y_grid, z_grid = np.meshgrid(x, y, z)
    
# 将网格转换为点云坐标
points = np.vstack([x_grid.ravel(), y_grid.ravel(), z_grid.ravel()]).T
print(f"points shape: {points.shape}")

x_steps: 84, y_steps: 69, z_steps: 66
points shape: (382536, 3)


In [22]:
z

array([-1.95993197, -1.92977917, -1.89962637, -1.86947357, -1.83932077,
       -1.80916797, -1.77901517, -1.74886237, -1.71870957, -1.68855677,
       -1.65840397, -1.62825117, -1.59809838, -1.56794558, -1.53779278,
       -1.50763998, -1.47748718, -1.44733438, -1.41718158, -1.38702878,
       -1.35687598, -1.32672318, -1.29657038, -1.26641758, -1.23626478,
       -1.20611198, -1.17595918, -1.14580638, -1.11565358, -1.08550078,
       -1.05534798, -1.02519518, -0.99504238, -0.96488959, -0.93473679,
       -0.90458399, -0.87443119, -0.84427839, -0.81412559, -0.78397279,
       -0.75381999, -0.72366719, -0.69351439, -0.66336159, -0.63320879,
       -0.60305599, -0.57290319, -0.54275039, -0.51259759, -0.48244479,
       -0.45229199, -0.42213919, -0.39198639, -0.36183359, -0.33168079,
       -0.301528  , -0.2713752 , -0.2412224 , -0.2110696 , -0.1809168 ,
       -0.150764  , -0.1206112 , -0.0904584 , -0.0603056 , -0.0301528 ,
       -0.        ])

In [23]:
# 找到深度最小的点所在的平面
min_z_index = -1  # 深度最小对应的索引
z_selected = z[-1]  # 深度最小的 z 平面
z_selected

-0.0

In [24]:
# 找到 x 和 y 的中心索引
center_x_index = x_steps // 2
center_y_index = y_steps // 2
center_y_index

34

In [25]:
# 计算 10×10 的范围
half_size = 10 // 2
x_start = max(center_x_index - half_size, 0)
x_end = min(center_x_index + half_size, x_steps)
y_start = max(center_y_index - half_size, 0)
y_end = min(center_y_index + half_size, y_steps)
print(x_start, x_end, y_start, y_end)

37 47 29 39


In [30]:
# 提取 10×10×10 点云
roi_x = x[x_start:x_end]
roi_y = y[y_start:y_end]
roi_z = z[-10:]  # 深度范围的 10 层
roi_z

array([-0.2713752, -0.2412224, -0.2110696, -0.1809168, -0.150764 ,
       -0.1206112, -0.0904584, -0.0603056, -0.0301528, -0.       ])

In [31]:
# 创建 10×10×10 的网格
roi_x_grid, roi_y_grid, roi_z_grid = np.meshgrid(roi_x, roi_y, roi_z)
roi_points = np.vstack([roi_x_grid.ravel(), roi_y_grid.ravel(), roi_z_grid.ravel()]).T
print(f"Extracted ROI points shape: {roi_points.shape}")

Extracted ROI points shape: (1000, 3)


### 经验证，center_x=center_y=0

In [ ]:




def extract_roi(points, spacing, roi_size=(10, 10, 10)):
    """
    从规则立方体点云中提取深度最小且靠近中心的 ROI 点云。
    
    :param points: 规则点云数据，形状为 (M, 3)。
    :param spacing: 每个点的间距，用于计算 ROI 的范围。
    :param roi_size: 提取的点云尺寸，默认 10x10x10。
    :return: ROI 点云数据，形状为 (N, 3)。
    """
    # 获取深度最小的 z 值
    min_z = points[:, 2].min()
    
    # 提取深度最小层的点
    min_z_points = points[points[:, 2] == min_z]
    
    # 找到宽度 (x) 和高度 (y) 的中心
    center_x = (min_z_points[:, 0].max() + min_z_points[:, 0].min()) / 2
    center_y = (min_z_points[:, 1].max() + min_z_points[:, 1].min()) / 2
    
    # 计算中心区域的范围
    half_size_x = roi_size[0] * spacing / 2
    half_size_y = roi_size[1] * spacing / 2
    
    roi_x_min = center_x - half_size_x
    roi_x_max = center_x + half_size_x
    roi_y_min = center_y - half_size_y
    roi_y_max = center_y + half_size_y
    
    # 提取满足范围的点
    roi_points = min_z_points[
        (min_z_points[:, 0] >= roi_x_min) & (min_z_points[:, 0] <= roi_x_max) &
        (min_z_points[:, 1] >= roi_y_min) & (min_z_points[:, 1] <= roi_y_max)
    ]
    
    # 如果点数量超出 10x10x10，取最近的点
    if roi_points.shape[0] > roi_size[0] * roi_size[1] * roi_size[2]:
        roi_points = roi_points[:roi_size[0] * roi_size[1] * roi_size[2]]
    
    print(f"Extracted ROI points: {roi_points.shape}")
    return roi_points

def save_roi_from_regular_cube(ray_points_camera, save_path, spacing=0.03, roi_size=(10, 10, 10)):
    """
    提取规则点云的 ROI 并保存为 .npy 文件。
    
    :param ray_points_camera: (N, max_steps, 3) 形状的张量，表示光线的坐标。
    :param save_path: .npy 文件保存路径。
    :param spacing: 每个点之间的间距。
    :param roi_size: 提取的 ROI 尺寸，默认 10x10x10。
    """
    # 生成规则立方体点云
    points = generate_regular_cube_points_from_ray(ray_points_camera, spacing)
    
    # 提取 ROI 点云
    roi_points = extract_roi(points, spacing, roi_size)
    
    # 保存为 .npy 文件
    np.save(save_path, roi_points)
    print(f"ROI point cloud has been saved to {save_path}")



In [5]:
points = generate_regular_cube_points_from_ray(ray_points_camera, spacing=0.03)

x_steps: 84, y_steps: 69, z_steps: 66
points shape: (382536, 3)
